In [2]:
! pip install cvxopt


You should consider upgrading via the 'c:\users\desmond\anaconda3\python.exe -m pip install --upgrade pip' command.



     --------------------------------------- 12.7/12.7 MB 75.2 kB/s eta 0:00:00


In [3]:
import numpy as np
import scipy as sp
import scipy.stats
from cvxopt import matrix, solvers
import pandas as pd

In [4]:
risk_free_rate = 0.06
dividend = 0.03
time_to_maturity = 1
volatility = 0.2
strike = 100
stock_price = 100
n_trials = 1000
n_steps = 200
func_list = [lambda x: x**0, lambda x: x]

In [6]:
! pip install MonteCarlo


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for MonteCarlo: filename=montecarlo-0.1.17-py3-none-any.whl size=1880 sha256=77c73026eb2498a00d8f2abc30b86629871e1cc396f139fa467049e580616201
  Stored in directory: c:\users\desmond\appdata\local\pip\cache\wheels\f2\91\9b\c056d13a912aae6b6a213331ee4c025ef1f3fbccc2f2db0b56
Successfully built MonteCarlo


You should consider upgrading via the 'c:\users\desmond\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
from MonteCarlo import MonteCarlo  

ModuleNotFoundError: No module named 'MonteCarlo'

In [ ]:
mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,dividend,volatility)
price_matrix = mc.simulate(n_trials,n_steps)

In [ ]:
mc.OHMCPricer()

In [ ]:
mc.MCPricer()

In [ ]:
mc.BSDeltaHedgedPricer()

In [ ]:
prices = mc.pricing(func_list=func_list)
print(prices)

In [ ]:
hedges = mc.hedging()
print(hedges)

In [ ]:
import timeit

In [ ]:
def performance_comparison(n_trials,n_steps):
    mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,dividend,volatility)
    # black scholes
    bs_start = timeit.default_timer()
    bs_price = mc.BlackScholesPricer('c')
    bs_sde_estimate = 0
    bs_end = timeit.default_timer()
    
    # regular MC
    rmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    rmc_price = mc.MCPricer('c')
    rmc_sde_estimate = mc.standard_error()
    rmc_end = timeit.default_timer()
    
    # regular MC with antithetic variates
    rmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps,antitheticVariates=True)
    rmc_anti_price = mc.MCPricer('c')
    rmc_anti_sde_estimate = mc.standard_error()
    rmc_anti_end = timeit.default_timer()
    
    # Delta-based MC
    dbmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    dbmc_price = mc.BSDeltaHedgedPricer('c')
    dbmc_sde_estimate = mc.standard_error()
    dbmc_end = timeit.default_timer()
    
    # Delta-based MC with antithetic variates
    dbmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps)
    dbmc_anti_price = mc.BSDeltaHedgedPricer('c')
    dbmc_anti_sde_estimate = mc.standard_error()
    dbmc_anti_end = timeit.default_timer()
    
    # OHMC
    ohmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    ohmc_price = mc.OHMCPricer('c')
    ohmc_sde_estimate = "NA"
    ohmc_end = timeit.default_timer()
    
    # OHMC with antithetic varietes
    ohmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps,antitheticVariates=True)
    ohmc_anti_price = mc.OHMCPricer('c')
    ohmc_anti_sde_estimate = "NA"
    ohmc_anti_end = timeit.default_timer()
    
    bs_runtime = bs_end - bs_start
    rmc_runtime = rmc_end - rmc_start
    dbmc_runtime = dbmc_end - dbmc_start
    ohmc_runtime = ohmc_end-ohmc_start
    rmc_anti_runtime = rmc_anti_end - rmc_anti_start
    dbmc_anti_runtime = dbmc_anti_end - dbmc_anti_start
    ohmc_anti_runtime = ohmc_anti_end-ohmc_anti_start
    
    rmc_err = np.abs(rmc_price - bs_price)
    dbmc_err = np.abs(dbmc_price - bs_price)
    ohmc_err = np.abs(ohmc_price - bs_price)
    rmc_anti_err = np.abs(rmc_anti_price - bs_price)
    dbmc_anti_err = np.abs(dbmc_anti_price - bs_price)
    ohmc_anti_err = np.abs(ohmc_anti_price - bs_price)
    
    
    result = {"method":["Black Scholes","MC","antithetic MC","DBMC","antithetic DBMC","OHMC","antithetic OHMC"],
              "runtime":[bs_runtime,rmc_runtime,rmc_anti_runtime,dbmc_runtime,dbmc_anti_runtime,
                         ohmc_runtime,ohmc_anti_runtime],
              "err":[0,rmc_err,rmc_anti_err,dbmc_err,dbmc_anti_err,ohmc_err,ohmc_anti_err], 
              "sde estimate":[0,rmc_sde_estimate,rmc_anti_sde_estimate,dbmc_sde_estimate,
                              dbmc_anti_sde_estimate,ohmc_sde_estimate,ohmc_anti_sde_estimate],
              "n_trials":[n_trials]*7,
              "n_steps":[n_steps]*7}
    return result

In [ ]:
n_trials_list = np.arange(100,1000,500)
n_steps_list = np.arange(50,200,50)
performance_df = pd.DataFrame()
for n_trials in n_trials_list:
    for n_steps in n_steps_list:
        print("n_trials: {}; n_steps: {}".format(n_trials,n_steps))
        new_df = pd.DataFrame(performance_comparison(n_trials,n_steps))
        performance_df = pd.concat((performance_df,new_df))

In [ ]:
sorted_performance_df = performance_df.set_index(['method','n_steps','n_trials']).sort_index()
sorted_performance_df

Variance reduction test

In [ ]:
# 
def performance_comparison2(n_trials,n_steps):
    mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,dividend,volatility)
    # black scholes
    bs_start = timeit.default_timer()
    bs_price = mc.BlackScholesPricer('c')
    bs_sde_estimate = 0
    bs_end = timeit.default_timer()
    
    # regular MC
    rmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    rmc_price = mc.MCPricer('c')
    rmc_sde_estimate = mc.standard_error()
    rmc_end = timeit.default_timer()
    
    # regular MC with antithetic variates
    rmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps,antitheticVariates=True)
    rmc_anti_price = mc.MCPricer('c')
    rmc_anti_sde_estimate = mc.standard_error()
    rmc_anti_end = timeit.default_timer()
    
    # OHMC
    ohmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    ohmc_price = mc.OHMCPricer('c')
    ohmc_sde_estimate = "NA"
    ohmc_end = timeit.default_timer()
    
    # OHMC with antithetic varietes
    ohmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps,antitheticVariates=True)
    ohmc_anti_price = mc.OHMCPricer('c')
    ohmc_anti_sde_estimate = "NA"
    ohmc_anti_end = timeit.default_timer()
    
    bs_runtime = bs_end - bs_start
    rmc_runtime = rmc_end - rmc_start
    ohmc_runtime = ohmc_end-ohmc_start
    rmc_anti_runtime = rmc_anti_end - rmc_anti_start
    ohmc_anti_runtime = ohmc_anti_end-ohmc_anti_start
    
    rmc_err = np.abs(rmc_price - bs_price)
    ohmc_err = np.abs(ohmc_price - bs_price)
    rmc_anti_err = np.abs(rmc_anti_price - bs_price)
    ohmc_anti_err = np.abs(ohmc_anti_price - bs_price)
    
    
    result = {"method":["Black Scholes","MC","antithetic MC","OHMC","antithetic OHMC"],
              "runtime":[bs_runtime,rmc_runtime,rmc_anti_runtime,ohmc_runtime,ohmc_anti_runtime],
              "err":[0,rmc_err,rmc_anti_err,ohmc_err,ohmc_anti_err], 
              "sde estimate":[0,rmc_sde_estimate,rmc_anti_sde_estimate,ohmc_sde_estimate,ohmc_anti_sde_estimate],
              "n_trials":[n_trials]*5,
              "n_steps":[n_steps]*5}
    return result

In [ ]:
n_trials_list = np.arange(100,10000,500)
# n_steps_list = np.arange(50,200,50)
performance_df2 = pd.DataFrame()
for n_trials in n_trials_list:
#     for n_steps in n_steps_list:
    print("n_trials: {}; n_steps: {}".format(n_trials,n_steps))
    new_df = pd.DataFrame(performance_comparison2(n_trials,n_steps))
    performance_df2 = pd.concat((performance_df2,new_df))

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

mc_trails = performance_df2[performance_df2["method"]=="MC"]["n_trials"]
mc_err = performance_df2[performance_df2["method"]=="MC"]["err"]
ax.plot(mc_trails,mc_err,label="regular MC")

anti_mc_trails = performance_df2[performance_df2["method"]=="antithetic MC"]["n_trials"]
anti_mc_err = performance_df2[performance_df2["method"]=="antithetic MC"]["err"]
ax.plot(anti_mc_trails,anti_mc_err,label="antithetic regular MC")

ohmc_trails = performance_df2[performance_df2["method"]=="OHMC"]["n_trials"]
ohmc_err = performance_df2[performance_df2["method"]=="OHMC"]["err"]
ax.plot(ohmc_trails,ohmc_err,label="OHMC")

anti_ohmc_trails = performance_df2[performance_df2["method"]=="antithetic OHMC"]["n_trials"]
anti_ohmc_err = performance_df2[performance_df2["method"]=="antithetic OHMC"]["err"]
ax.plot(anti_ohmc_trails,anti_ohmc_err,label="antithetic OHMC")

ax.legend()
plt.xlabel("n_trials")
plt.ylabel("err")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

mc_runtime = performance_df2[performance_df2["method"]=="MC"]["runtime"]
mc_err = performance_df2[performance_df2["method"]=="MC"]["err"]
ax.scatter(mc_runtime,mc_err,label="regular MC")

anti_mc_runtime = performance_df2[performance_df2["method"]=="antithetic MC"]["runtime"]
anti_mc_err = performance_df2[performance_df2["method"]=="antithetic MC"]["err"]
ax.scatter(mc_runtime,mc_err,label="antithetic regular MC")

ohmc_runtime = performance_df2[performance_df2["method"]=="OHMC"]["runtime"]
ohmc_err = performance_df2[performance_df2["method"]=="OHMC"]["err"]
ax.scatter(ohmc_runtime,ohmc_err,label="OHMC")

anti_ohmc_runtime = performance_df2[performance_df2["method"]=="antithetic OHMC"]["runtime"]
anti_ohmc_err = performance_df2[performance_df2["method"]=="antithetic OHMC"]["err"]
ax.scatter(ohmc_runtime,ohmc_err,label="antithetic OHMC")

ax.legend()
plt.xlabel("runtime")
plt.ylabel("err")

plt.show()